# Simulation fo the behaviour of the call center model with service delay

In [ ]:
using Random
using DelimitedFiles
using PrettyTables

# Parameters
horiz = 20                    # total thorizon
n_ser = 2                    # number of servers
XM = 10                       # max queue length before dropping calls

d_fn = "..//CC_simple//d_mat_Thu_22_May_2025_19_39_50.txt";
a_fn = "..//CC_simple//a_mat_Thu_22_May_2025_19_39_50.txt";

d_mat = DelimitedFiles.readdlm(d_fn);
a_mat = DelimitedFiles.readdlm(a_fn);

d = d_mat[1:horiz, 1]  # demand matrix for incoming calls
a = a_mat[1:horiz, 1]  # abandonment matrix for calls

t = 1:horiz

## Short version original problem

In [ ]:
# State variables
X = zeros(Int, horiz+1)     # current number of customers in queue x(k)
s = zeros(Int, horiz+1)     # number of served customers per timestep
dr = zeros(Int, horiz)      # dropped due to full buffer
z = zeros(Int, horiz+1)     # custumers server 

X[1] = 3  # initial state, no customers in queue
s[1] = n_ser
z[1] = 0


for k in 1:horiz
    X[k+1] = X[k] + d[k] - s[k] - a[k]

    # If queue exceeds max length, drop excess
    if X[k+1] > XM
        dr[k] = X[k+1] - XM  # dropped calls due to full queue
        X[k+1] = XM
    else
        dr[k] = 0
    end
    
    z[k+1] = z[k] + s[k]
    # Serve up to number of servers
    s[k+1] = min(X[k], n_ser)
end

In [46]:
data = hcat(t, X[1:horiz,1], s[1:horiz,1], dr, d_mat[:,1], a_mat[:,1])
header = (["t", "X", "s", "dr", "d_mat", "a_mat"]);

println("Results")
pretty_table(data; header=header,formatters=ft_printf("%5.3f",1:11))


Results
┌────────┬────────┬───────┬────────┬────────┬───────┐
│      t │      X │     s │     dr │  d_mat │ a_mat │
├────────┼────────┼───────┼────────┼────────┼───────┤
│  1.000 │  3.000 │ 2.000 │  0.000 │  7.000 │ 0.000 │
│  2.000 │  8.000 │ 2.000 │  0.000 │  3.000 │ 1.000 │
│  3.000 │  8.000 │ 2.000 │  0.000 │  3.000 │ 2.000 │
│  4.000 │  7.000 │ 2.000 │  1.000 │  7.000 │ 1.000 │
│  5.000 │ 11.000 │ 2.000 │ 10.000 │ 12.000 │ 1.000 │
│  6.000 │ 20.000 │ 2.000 │ 15.000 │  9.000 │ 2.000 │
│  7.000 │ 25.000 │ 2.000 │ 18.000 │  6.000 │ 1.000 │
│  8.000 │ 28.000 │ 2.000 │ 18.000 │  3.000 │ 1.000 │
│  9.000 │ 28.000 │ 2.000 │ 15.000 │  0.000 │ 1.000 │
│ 10.000 │ 25.000 │ 2.000 │ 21.000 │  9.000 │ 1.000 │
│ 11.000 │ 31.000 │ 2.000 │ 29.000 │ 11.000 │ 1.000 │
│ 12.000 │ 39.000 │ 2.000 │ 38.000 │ 12.000 │ 1.000 │
│ 13.000 │ 48.000 │ 2.000 │ 46.000 │ 12.000 │ 2.000 │
│ 14.000 │ 56.000 │ 2.000 │ 54.000 │ 12.000 │ 2.000 │
│ 15.000 │ 64.000 │ 2.000 │ 54.000 │  2.000 │ 0.000 │
│ 16.000 │ 64.000 │ 

## Short version with delay

In [50]:
# State variables
X_dl = zeros(Int, horiz+1)     # current number of customers in queue x(k)
s_dl = zeros(Int, horiz+1)     # number of served customers per timestep
dr_dl = zeros(Int, horiz)      # dropped due to full buffer
z_dl = zeros(Int, horiz+1)     # custumers server 

X_dl[1] = 3  # initial state, no customers in queue
s_dl[1] = 0
z_dl[1] = 0

# Servers
t_ser = 4 # time to serve one customer 
st_ser = zeros(Int, n_ser) # state of servers
S = zeros(n_ser, t_ser) # servers conveyor

# another option would be to S as
# S = zeros(Int, n_ser) # state of servers, where 0 means free and >0 means busy with remaining time

for k in 1:horiz

    for i in 1:n_ser
        st_ser[i] = S[i, 1] # get the state of server i
        S[i, 2:t_ser] = S[i, 1:t_ser-1] # shift the state of server i

        # If server is free, start serving a customer
        if st_ser[i] == 0 && X_dl[k] > 0
            s_dl[k] += 1
            S[i, 1] = 1
        else 
            S[i, 1] = 0
        end
    end

    X_dl[k+1] = X_dl[k] + d[k] - s_dl[k] -a[k]

    # If queue exceeds max length, drop excess
    if X_dl[k+1] > XM
        dr_dl[k] = X_dl[k+1] - XM  # dropped calls due to full queue
        X_dl[k+1] = XM
    else
        dr_dl[k] = 0
    end

    # Served custumers counted only after the service time
    if k <= t_ser
        z_dl[k+1] = 0
    else
        z_dl[k+1] = z_dl[k] + s_dl[k-t_ser]
    end

end

In [51]:
data = hcat(t, X_dl[1:horiz,1], s_dl[1:horiz,1], dr, z_dl[1:horiz,1], d_mat[:,1], a_mat[:,1])
header = (["t", "X_dl", "s_dl", "dr", "z_dl", "d_mat", "a_mat"]);

println("Results")
pretty_table(data; header=header,formatters=ft_printf("%5.3f",1:11))

Results
┌────────┬────────┬───────┬────────┬────────┬────────┬───────┐
│      t │   X_dl │  s_dl │     dr │   z_dl │  d_mat │ a_mat │
├────────┼────────┼───────┼────────┼────────┼────────┼───────┤
│  1.000 │  3.000 │ 2.000 │  0.000 │  0.000 │  7.000 │ 0.000 │
│  2.000 │  8.000 │ 0.000 │  0.000 │  0.000 │  3.000 │ 1.000 │
│  3.000 │ 10.000 │ 2.000 │  0.000 │  0.000 │  3.000 │ 2.000 │
│  4.000 │  9.000 │ 0.000 │  1.000 │  0.000 │  7.000 │ 1.000 │
│  5.000 │ 10.000 │ 2.000 │  9.000 │  0.000 │ 12.000 │ 1.000 │
│  6.000 │ 10.000 │ 0.000 │  5.000 │  2.000 │  9.000 │ 2.000 │
│  7.000 │ 10.000 │ 2.000 │  3.000 │  2.000 │  6.000 │ 1.000 │
│  8.000 │ 10.000 │ 0.000 │  0.000 │  4.000 │  3.000 │ 1.000 │
│  9.000 │ 10.000 │ 2.000 │  0.000 │  4.000 │  0.000 │ 1.000 │
│ 10.000 │  7.000 │ 0.000 │  3.000 │  6.000 │  9.000 │ 1.000 │
│ 11.000 │ 10.000 │ 2.000 │  8.000 │  6.000 │ 11.000 │ 1.000 │
│ 12.000 │ 10.000 │ 0.000 │  9.000 │  8.000 │ 12.000 │ 1.000 │
│ 13.000 │ 10.000 │ 2.000 │  8.000 │  8.000 │ 1

## Short version with delay and server ocupied 

In [85]:
# State variables
X_dl = zeros(Int, horiz+1)     # current number of customers in queue x(k)
s_dl = zeros(Int, horiz)     # number of customers enering server per timestep
s_end = zeros(Int, horiz)    # number of served customers per the time step
dr_dl = zeros(Int, horiz)      # dropped due to full buffer
z_dl = zeros(Int, horiz+1)     # custumers server 

X_dl[1] = 3  # initial state, no customers in queue
s_dl[1] = 0
z_dl[1] = 0

# Servers
t_ser = [2, 3] # delay to serve one customer 
st_ser = zeros(n_ser, horiz+1) # state of servers, where 0 means free and >0 means busy with remaining time


for k in 1:horiz

    for i in 1:n_ser

        # If server is free, start serving a customer
        if st_ser[i, k] == 0 && X_dl[k] > 0
            s_dl[k] += 1
            st_ser[i, k+1] = t_ser[i]

            if k > 1 
                s_end[k] += 1
            end
        else 
            st_ser[i, k+1] = st_ser[i, k] - 1  # decrement the remaining service time
        end
    end

    X_dl[k+1] = X_dl[k] + d[k] - s_dl[k] -a[k]

    # If queue exceeds max length, drop excess
    if X_dl[k+1] > XM
        dr_dl[k] = X_dl[k+1] - XM  # dropped calls due to full queue
        X_dl[k+1] = XM
    else
        dr_dl[k] = 0
    end
        
    z_dl[k+1] = z_dl[k] + s_end[k]

end

In [86]:
g = [2, 5]
f = zeros(Int, 2)
println(g[1])

2


In [87]:
data = hcat(t, X_dl[1:horiz,1], s_dl[1:horiz,1], s_end, st_ser[1, 1:horiz], st_ser[2, 1:horiz],  dr, z_dl[1:horiz,1], d_mat[:,1], a_mat[:,1])
header = (["t", "X_dl", "s_dl", "s_end", "st_ser1", "st_ser2", "dr", "z_dl", "d_mat", "a_mat"]);

println("Results")
pretty_table(data; header=header,formatters=ft_printf("%5.3f",1:11))

Results
┌────────┬────────┬───────┬───────┬─────────┬─────────┬────────┬────────┬────────┬───────┐
│      t │   X_dl │  s_dl │ s_end │ st_ser1 │ st_ser2 │     dr │   z_dl │  d_mat │ a_mat │
├────────┼────────┼───────┼───────┼─────────┼─────────┼────────┼────────┼────────┼───────┤
│  1.000 │  3.000 │ 2.000 │ 0.000 │   0.000 │   0.000 │  0.000 │  0.000 │  7.000 │ 0.000 │
│  2.000 │  8.000 │ 0.000 │ 0.000 │   2.000 │   3.000 │  0.000 │  0.000 │  3.000 │ 1.000 │
│  3.000 │ 10.000 │ 0.000 │ 0.000 │   1.000 │   2.000 │  0.000 │  0.000 │  3.000 │ 2.000 │
│  4.000 │ 10.000 │ 1.000 │ 1.000 │   0.000 │   1.000 │  1.000 │  0.000 │  7.000 │ 1.000 │
│  5.000 │ 10.000 │ 1.000 │ 1.000 │   2.000 │   0.000 │  9.000 │  1.000 │ 12.000 │ 1.000 │
│  6.000 │ 10.000 │ 0.000 │ 0.000 │   1.000 │   3.000 │  5.000 │  2.000 │  9.000 │ 2.000 │
│  7.000 │ 10.000 │ 1.000 │ 1.000 │   0.000 │   2.000 │  3.000 │  2.000 │  6.000 │ 1.000 │
│  8.000 │ 10.000 │ 0.000 │ 0.000 │   2.000 │   1.000 │  0.000 │  3.000 │  3.000 │